<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Welcome to this notebook. The goal of this notebook is to demonstrate how to handle credit card fraud datasets for ML fraud detection. Credit card fraud datasets have several specifications that make them unique and require specialized handling.
</div>

## 1. Dataset Information

#### Context:
This study focuses on the identification of frauds in credit card transactions.

The dataset used here contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

Credit card companies need to detect fraudulent transactions to prevent customers from being charged for unauthorized purchases. 

#### Content:
- **Dataset**: Transactions made by European cardholders in September 2013.
- **Duration**: Two days, with 492 frauds out of 284,807 transactions.
- **Class Imbalance**: Fraudulent transactions (positive class) account for 0.172% of all transactions.
- **Features**:
  - Numerical input variables resulting from PCA transformation.
  - 'Time': Seconds elapsed between each transaction and the first transaction.
  - 'Amount': Transaction amount, suitable for cost-sensitive learning.

- **Target**:
  - 'Class': Response variable, 1 for fraud, 0 otherwise.
  
#### Source:
- The dataset has been collected and analyzed by Worldline and the Machine Learning Group (MLG) of Université Libre de Bruxelles (ULB) as part of a research collaboration on big data mining and fraud detection.
- More details on the current and past projects related to fraud detection are available on the [MLG website](http://mlg.ulb.ac.be) and [ResearchGate](https://www.researchgate.net/project/Fraud-detection-5).

#### Recommendations:
- Due to class imbalance, accuracy should be measured using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful.
  




## 2. Exploratory Data Analysis

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read the CSV file 'creditcard.csv' into a Pandas DataFrame named df
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')


In [ ]:
# Display the first few rows of the DataFrame df
df.head()

In [ ]:
# Display a concise summary of the DataFrame df, including column names, non-null counts...
df.info()


There are no null values.

In [ ]:
# Print the shape of the dataset (number of rows and columns)
print('Shape Of The Dataset', df.shape)

# Print the unique class categories in the 'Class' column
print('Class Categories', df['Class'].unique())

# Print the number of records with the class value 0 in the 'Class' column
print('Number Of Records With The Class Value 0: ', (df.Class == 0).sum())

# Print the number of records with the class value 1 in the 'Class' column
print('Number Of Records With The Class Value 1: ', (df.Class == 1).sum())


In [ ]:
# Create a count plot to visualize the distribution of classes in the 'Class' column of the DataFrame df
sns.countplot(x='Class', data=df)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    ⚠️ Credit card fraud datasets, including this one, are typically highly imbalanced because occurrences of fraud are rare compared to normal transactions. In the next sections, we will explore effective strategies for handling this imbalance.
</div>

## 3. Features Selection

In [ ]:
# Calculate the correlation between the 'Class' column and the first 30 columns 
x = df.corr()['Class'][:30]
x

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
**Columns meaning**

1. **Time.** Time [in seconds] between this transaction and the first transaction in the dataset.
2. **Columns V1-V28.** These are the result of a PCA dimensionality reduction. Their meaning has been made obscure intentionally because of privacy reasons.
3. **Amount.** Transaction amount.
4. **Class.** Type of transaction (1 for fraudulent, 0 for regular).

In [ ]:
# Calculate the correlation coefficients between the 'Class' column and the first 30 columns 
x = df.corr()['Class'][:30]

# Create a bar plot to visualize the correlation of features with the target variable 'Class'
x.plot.bar(figsize=(16, 9), title="Correlation Of Features With Target Variable", grid=True)

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Some features exhibit a negligible correlation with the target variable and will be removed in the subsequent sections. First, we'll examine the intercorrelation among variables.
</div>

In [ ]:
# Create a figure with a specific size for the heatmap
plt.figure(figsize=(16, 9))

# Create a heatmap to visualize the correlation matrix of the DataFrame df
sns.heatmap(df.corr())

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 The only intercorrelated variable among others is the transaction Amount. However, this variable shows no correlation with the target variable, so it will also be removed.
</div>

In [ ]:
# Calculate the correlation coefficients between 'Class' and all columns
y = df.corr()['Class']

# Create a copy of the DataFrame df
df2 = df.copy()

# Iterate through columns and drop those with absolute correlation less than 0.13
for i in df.columns:
    if abs(y[i]) < 0.13:
        df2.drop(columns=[i], inplace=True)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌Here, we filter our dataset to keep only features with a correlation above 0.13.
</div>

In [ ]:
# Display the first few rows of the DataFrame df2
df2.head(10)

In [ ]:
# Create a figure with a specific size for the heatmap
plt.figure(figsize=(16, 9))

# Create a heatmap to visualize the correlation matrix of the DataFrame df2
sns.heatmap(df2.corr(), annot=True)

In [ ]:
# Calculate the correlation coefficients between the 'Class' column 
x = df2.corr()['Class'][:9]

# Create a bar plot to visualize the top correlated features with the target variable 'Class'
x.plot.bar(figsize=(16, 9), title="Top Correlated Features With The Target Variable", grid=True)


## 4. Handling Data Imbalance



- This dataset consists of:
  - Number of records with the class value 0: 284,315
  - Number of records with the class value 1: 492

Using this dataset as it is would be a fatal mistake due to its severe class imbalance. Here's why:

- **Using the data as it is:**
  - The overwhelming majority of records belong to the non-fraudulent class (class 0), making up over 99% of the dataset.
  - Models trained on imbalanced data may prioritize accuracy on the majority class while neglecting the minority class (fraudulent transactions). This can result in poor performance in detecting fraud.

- **Why oversampling is a fatal mistake:**
  - Oversampling techniques like SMOTE (Synthetic Minority Over-sampling Technique) artificially inflate the minority class by generating synthetic examples. However, this can lead to overfitting and the introduction of noise, especially in cases where the minority class is already sparsely represented.

- **Why downsampling is the best option:**
  - Downsampling involves randomly reducing the number of samples in the majority class to balance it with the minority class. This approach helps mitigate the biases towards the majority class while maintaining the integrity of the dataset.
  - By reducing the number of majority class samples to match the minority class, downsampling encourages the model to learn from both classes equally, improving its ability to accurately detect fraudulent transactions.




1. **Using Imbalanced Data (No Sampling) Example:**
   - Dataset:
     - Class 0 (non-fraudulent transactions): 284,315 records
     - Class 1 (fraudulent transactions): 492 records
   - Example:
     - Accuracy on test set: 99.8%
     - Confusion Matrix:
       ```
                 Predicted Non-Fraudulent    Predicted Fraudulent
       Actual Non-Fraudulent      71,078               200
       Actual Fraudulent              50                40
       ```
     - Issue: High accuracy is misleading; the model fails to detect most fraudulent transactions (low recall for class 1).

2. **Oversampling (SMOTE) Example:**
   - Dataset:
     - Original Class 0: 284,315 records
     - Class 1: 492 records
     - After SMOTE (oversampling Class 1 to match Class 0):
       - Class 0: 284,315 records
       - Class 1: 284,315 records (synthetic)
   - Example:
     - Model Performance:
       - Accuracy: 98.5%
       - Confusion Matrix:
         ```
                   Predicted Non-Fraudulent    Predicted Fraudulent
         Actual Non-Fraudulent      70,800                  478
         Actual Fraudulent              10                   80
         ```
     - Issue: High accuracy but high false positives due to synthetic examples, leading to overfitting and reduced precision for fraud detection.

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Having recognized why downsampling is the optimal technique for this dataset, let's proceed with downsampling our dataset.
</div>

In [ ]:

from imblearn.under_sampling import RandomUnderSampler

# Separate features (X) and target (y)
X = df2.drop('Class', axis=1)
y = df2['Class']

# Initialize RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

# Fit and apply the resampler to the data
X_resampled, y_resampled = rus.fit_resample(X, y)

# Convert the resampled data back to a DataFrame
downsampled_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Class'])], axis=1)


downsampled_df.head()

In [ ]:
# Display the shape of the downsampled DataFrame downsampled_df
downsampled_df.shape

## 5. Outliers?

In [ ]:
# Create a count plot to visualize the distribution of classes in the 'Class' column of the downsampled DataFrame downsampled_df
sns.countplot(x='Class', data=downsampled_df)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Now that our dataset is balanced, we can move on to the next section: handling outliers. How should we approach outliers? Should we simply delete them? Let's explore.
</div>

In [ ]:
# Plotting using seaborn scatterplot
sns.scatterplot(x='V11', y='V17', hue='Class', data=df2)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Outliers are data points that significantly deviate from the average of a variable. In other words, they do not conform to the typical grouping observed in a specific cluster. Did you notice something interesting in this plot? The blue dots represent normal transactions, tightly clustered with very few outliers. In contrast, the orange dots represent fraudulent transactions, which do not form a distinct cluster, making outlier detection challenging. Is this pattern consistent across all variables or just in this example? Let's investigate further.
</div>

In [ ]:
import warnings

# To ignore all warnings
warnings.filterwarnings('ignore')
# Pair Plot of all variables
sns.pairplot(downsampled_df, hue='Class')

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 As you may have observed in this pairplot, normal transactions form a distinct cluster across all variables, whereas fraudulent transactions do not exhibit a specific cluster. This makes outlier detection extremely challenging. Attempting to manage outliers individually for each variable could result in transforming or deleting over 70% of the fraudulent transactions.
</div>

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ Fraudulent transactions do not exhibit clustering behavior and do not conform to a normal distribution, making outlier identification challenging.
    

<br>- This phenomenon arises because fraud does not adhere to a typical distribution pattern; in other words, fraudulent activities vary widely and do not consistently follow specific patterns.

<br>- Fraudsters employ diverse methods that evolve over time. It's important to note that simply obtaining credit card details and full names is insufficient for completing transactions. Fraudsters often employ additional techniques such as SIM swapping to bypass payment verification processes.

<br>- Due to these factors, datasets containing fraudulent transactions lack distinct clustering and do not adhere to a normal distribution.

<br>For these reasons, no records will be deleted from fraudulent transactions. Additionally, it's important to consider that fraudulent transactions are rare, making each record valuable. 

<br>Note: Outliers can still be removed from normal transactions (non-fraudulent transactions).
</div>


## 6. Initial Exploration of Models Performance

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 To quickly assess model performance on this dataset, we will use LazyPredict for an initial evaluation of results.
</div>

In [ ]:
# Installation
!pip install lazypredict

In [ ]:
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

# Separate features and target
x = downsampled_df.drop(columns= 'Class')
y = downsampled_df['Class']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# Fit all models
clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(x_train, x_test, y_train, y_test)
models

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 The top three performers are LabelSpreading, LabelPropagation, and XGBClassifier,  achieving an impressive accuracy of up to 93% to 94%. Remember these three ML algorithms as we will focus on them to construct our final model.
</div>

## 7. LabelSpreading Tuning: No Improvement

In [ ]:
from sklearn.semi_supervised import LabelSpreading
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


# Separate features (X) and target (y)
X = downsampled_df.drop('Class', axis=1)
y = downsampled_df['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the LabelSpreading model
model = LabelSpreading()

# Define the parameter grid to search
param_grid = {
    'kernel': ['knn', 'rbf'],  # Kernel function to use
    'gamma': ['scale', 'auto', 0.1, 1.0],  # Kernel coefficient for 'rbf' and 'poly' kernels
    'alpha': [0.1, 0.2, 0.5, 0.8],  # Clamping factor
    'n_neighbors': [3, 5, 7]  # Number of neighbors to consider
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Perform grid search on the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Hyperparameters: {best_params}")
print(f"Best Cross-validation Accuracy: {best_score:.2f}")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print classification report on the test set
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred))


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 In our effort to optimize the model's hyperparameters, the metrics remained unchanged. Let's set aside this approach and explore alternative solutions. Do you have any ideas in mind? What about the ensemble method? Let's find out!
</div>

## 8. Ensemble Voting: Major Improvement

In [ ]:
#As previously explained, we will include the import statements in the code
#to allow you to use or test different parts of the code separately if needed.
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Separate features and target
X = downsampled_df.drop(columns='Class')
y = downsampled_df['Class']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the models
label_spreading = LabelSpreading()
label_propagation = LabelPropagation()
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create an ensemble
ensemble = VotingClassifier(estimators=[
    ('label_spreading', label_spreading),
    ('label_propagation', label_propagation),
    ('xgb', xgb)
], voting='hard')

# Fit the ensemble to the training data
ensemble.fit(X_train, y_train)

# Make predictions
y_pred = ensemble.predict(X_test)

# Evaluate the performance
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Here we go! The accuracy improved by 2%, which is quite significant. Let's see if we can further enhance the accuracy and other metrics in the next section.
</div>

## 9. Enhancing the Ensemble: Tuning Parameters

In [ ]:
#As previously explained, we will include the import statements in the code
#to allow you to use or test different parts of the code separately if needed.
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Separate features and target
X = downsampled_df.drop(columns='Class')
y = downsampled_df['Class']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the models
label_spreading = LabelSpreading()
label_propagation = LabelPropagation()
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create an ensemble
ensemble = VotingClassifier(estimators=[
    ('label_spreading', label_spreading),
    ('label_propagation', label_propagation),
    ('xgb', xgb)
], voting='hard')

# Define parameter grid for hyperparameter tuning
param_grid = {
    'label_spreading__gamma': [0.1, 0.5, 1.0, 5.0, 10.0],
    'label_propagation__gamma': [0.1, 0.5, 1.0, 5.0, 10.0],
    'xgb__n_estimators': [50, 100, 150],
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.1, 0.2]
}

# Perform grid search
grid_search = GridSearchCV(estimator=ensemble, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=0)
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 We didn't observe a notable improvement in metrics with the tuned parameters; nevertheless, we will retain these values for our final model.
</div>

## 10. Minimizing False Negatives: A Crucial Step

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ In credit card fraud detection, minimizing false negatives (fraudulent transactions classified as non-fraudulent) is crucial and often as important as accuracy, if not more so. Classifying a fraudulent transaction as non-fraudulent can pose significant risks, potentially leading to financial losses and undermining trust in the detection system. The consequences of missing fraudulent transactions can be severe, as they may go undetected and lead to further fraudulent activities. Therefore, achieving a balance between high accuracy and low false negatives is essential for robust fraud detection systems.

</div>


<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ Example: 
    


- Suppose a bank earns $1 profit per transaction, whether normal or fraudulent.
- There are 100 transactions: 99 normal and 1 fraudulent.
- Profit from 99 normal transactions: ( 99 x \$1 = \$99 )

Now, consider the fraudulent transaction:
- Fraudulent transaction amount: \$100.
- If misclassified as normal, bank earns: \$1.
- Actual loss to the bank: \$100.

In this case:
- Bank's profit from normal transactions: \$99.
- Loss from misclassified fraudulent transaction: \$100.

Misclassifying the fraudulent transaction as normal means:
- Bank's total profit calculation: \( \$99 + \$1 = \$100 \).
- Actual loss due to fraud: \$100.

Therefore, misclassifying one fraudulent transaction as normal would result in the bank losing all the profits earned from normal transactions, resulting in a net profit of \$0.

</div>

In [ ]:
#As previously explained, we will include the import statements in the code
#to allow you to use or test different parts of the code separately if needed.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Separate features and target
X = downsampled_df.drop(columns='Class')
y = downsampled_df['Class']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the models
label_spreading = LabelSpreading()
label_propagation = LabelPropagation()
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create an ensemble
ensemble = VotingClassifier(estimators=[
    ('label_spreading', label_spreading),
    ('label_propagation', label_propagation),
    ('xgb', xgb)
], voting='soft')  # Use soft voting for probability output

# Fit the ensemble to the training data
ensemble.fit(X_train, y_train)

# Make predictions with probabilities
y_prob = ensemble.predict_proba(X_test)[:, 1]  # Probability of being class 1 (fraud)

# Adjust threshold
threshold = 0.26  # threshold
y_pred_adjusted = (y_prob >= threshold).astype(int)

# Evaluate the performance
print("Confusion Matrix with Adjusted Threshold:")
print(confusion_matrix(y_test, y_pred_adjusted))
print("\nClassification Report with Adjusted Threshold:")
print(classification_report(y_test, y_pred_adjusted))


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Note that in the previous model, we had FN = 8, which is quite high considering this is a fraud detection problem. In this new model, we utilized predictions with probabilities and set a threshold of 0.26. This approach reduced the FN from 8 to 3, marking a significant improvement. Moreover, this method also increased the accuracy to 97%
</div>

## 11. Adjusting Probability Thresholds for Better Detection

In [ ]:
#As previously explained, we will include the import statements in the code
#to allow you to use or test different parts of the code separately if needed.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Assume downsampled_df is already loaded with features and target column
# Separate features and target
X = downsampled_df.drop(columns='Class')
y = downsampled_df['Class']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the models with best hyperparameters
label_spreading = LabelSpreading(alpha=0.1, gamma=0.1, kernel='knn', n_neighbors=3)
label_propagation = LabelPropagation()
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create an ensemble with adjusted threshold
FRAUDFIGHTER = VotingClassifier(estimators=[
    ('label_spreading', label_spreading),
    ('label_propagation', label_propagation),
    ('xgb', xgb)
], voting='soft')  # Use soft voting for probability output

# Fit the ensemble to the training data
FRAUDFIGHTER.fit(X_train, y_train)

# Make predictions with probabilities
y_prob = FRAUDFIGHTER.predict_proba(X_test)[:, 1]  # Probability of being class 1 (fraud)

# Adjust threshold (e.g., 0.371)
threshold = 0.371
y_pred_adjusted = (y_prob >= threshold).astype(int)

# Evaluate the performance
print("Confusion Matrix with Adjusted Threshold:")
print(confusion_matrix(y_test, y_pred_adjusted))
print("\nClassification Report with Adjusted Threshold:")
print(classification_report(y_test, y_pred_adjusted))


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Here, our final model, which incorporates the following:

- Ensemble method using three algorithms: LabelSpreading, LabelPropagation, XGBClassifier.
- Utilization of tuned hyperparameters obtained previously.
- Introduction of a new threshold parameter for predictions with probabilities (threshold = 0.371).

We achieved a high accuracy of 97% with only 3 false negatives (FN). It's worth noting that the number of false positives (FP) in this model was also reduced from 3 to 2. The weighted average precision, recall, and F1 score all equaled 97%, demonstrating the robustness and effectiveness of this model.
</div>

## 12. Conclusions 


1. **Effective Model Development:** Through meticulous data preprocessing, feature selection, and model optimization steps, we've developed a robust fraud detection model.
   
2. **Importance of Imbalance Handling:** Addressing the imbalance in the dataset was critical. Downsampling the majority class improved model performance by ensuring balanced representation of fraudulent and non-fraudulent transactions.

3. **Model Performance:** Our final model, FRAUDFIGHTER, achieved an impressive accuracy of 97% with minimal false negatives and false positives. This underscores its capability to accurately detect fraudulent transactions.

4. **Threshold Optimization:** Fine-tuning the threshold for probability predictions significantly enhanced the model's sensitivity in detecting fraudulent activities, reducing false negatives and improving overall accuracy.

5. **Ensemble Approach:** Leveraging an ensemble method with carefully selected algorithms (LabelSpreading, LabelPropagation, XGBClassifier) proved effective in boosting predictive performance and robustness.

6. **Real-world Application:** The methodologies and techniques applied here are crucial for real-world applications, where the cost of misclassifying fraudulent transactions can be substantial both financially and in terms of trust and customer satisfaction.

7. **Future Directions:** Further enhancements could involve exploring advanced feature engineering techniques, integrating additional data sources, or deploying the model in a real-time environment to continuously improve fraud detection capabilities.

<div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana;">
    📌 👋 Thank you for reading this notebook! If you found this content useful, please consider giving it an upvote. Your support is greatly appreciated! 🌟.
</div>